In [1]:
import torch
import torch.nn as nn
import torchvision #for dataset 
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\tahah\anaconda3\envs\riemann\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] La procédure spécifiée est introuvable'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
input_size = 28*28
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
lr = 0.01

In [3]:
class PortfolioRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(PortfolioRNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        # x -> (batch_size, timesteps/seq, input_size/feature_size)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, X, r):
        h0 = torch.zeros(self.num_layers, X.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(X,h0)
        # out -> (batch_size, seq_length, hidden_size)
        #out = out[:,-1,:] only take the last time step NO WE NEED ALL TIME STEPS
        out = self.linear(out)
        out = torch.softmax(out, dim=-1)
        portfolio_returns = torch.sum(out[:, :, :] * r[:, :, :], dim=-1)
        # portfolio_returns -> (batch_size)
        sharpe = torch.mean(portfolio_returns, dim = -1) / torch.std(portfolio_returns, dim = -1)
        return -sharpe.mean()
    
    def get_allocations(self, X):
        with torch.no_grad():
            h0 = torch.zeros(self.num_layers, X.size(0), self.hidden_size).to(device)
            out, _ = self.rnn(X,h0)
            out = self.linear(out)
            out = torch.softmax(out, dim=-1)
        return out[:, -1, :]

In [4]:
import numpy as np
import yfinance as yf
import pandas as pd

tickers = ['VTI', 'AGG', 'DBC', '^VIX']
data = yf.download(tickers, start="2006-01-01", end="2020-12-31", interval="1d")['Adj Close']
data_na = data.dropna(axis = 0)
for column in data_na.columns:
    data_na[f'{column}_R'] = data_na[f'{column}'].pct_change()
    #le rendement du jour suivant
    data_na[f'{column}_y'] = data_na[f'{column}_R'].shift(-1)
data_na.dropna(axis=0, inplace=True)
data_na.reset_index(inplace=True)
data_na['Date'] = data_na['Date'].dt.date
data_na

[*********************100%***********************]  4 of 4 completed
C:\Users\tahah\AppData\Local\Temp\ipykernel_10784\376173755.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_na[f'{column}_R'] = data_na[f'{column}'].pct_change()
C:\Users\tahah\AppData\Local\Temp\ipykernel_10784\376173755.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_na[f'{column}_y'] = data_na[f'{column}_R'].shift(-1)
C:\Users\tahah\AppData\Local\Temp\ipykernel_10784\376173755.py:9: SettingWithCopyWarning: 
A value is

Ticker,Date,AGG,DBC,VTI,^VIX,AGG_R,AGG_y,DBC_R,DBC_y,VTI_R,VTI_y,^VIX_R,^VIX_y
0,2006-02-07,56.438538,20.285255,44.219498,13.590000,-0.000699,-0.000499,-0.028926,-0.004255,-0.009736,0.007194,0.042178,-0.055923
1,2006-02-08,56.410393,20.198933,44.537632,12.830000,-0.000499,0.000600,-0.004255,0.009402,0.007194,-0.001905,-0.055923,0.022603
2,2006-02-09,56.444248,20.388842,44.452785,13.120000,0.000600,-0.002098,0.009402,-0.018205,-0.001905,0.002067,0.022603,-0.019055
3,2006-02-10,56.325848,20.017662,44.544689,12.870000,-0.002098,0.000700,-0.018205,-0.015524,0.002067,-0.004523,-0.019055,0.037296
4,2006-02-13,56.365253,19.706907,44.343197,13.350000,0.000700,-0.001599,-0.015524,-0.008322,-0.004523,0.009486,0.037296,-0.082397
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3745,2020-12-22,106.759193,13.624199,181.857010,24.230000,0.001443,-0.000678,-0.011004,0.012517,0.000207,0.001710,-0.036963,-0.037969
3746,2020-12-23,106.686768,13.794738,182.168076,23.309999,-0.000678,0.000933,0.012517,0.002747,0.001710,0.001673,-0.037969,-0.076362
3747,2020-12-24,106.786339,13.832635,182.472778,21.530001,0.000933,0.000170,0.002747,-0.006164,0.001673,0.006327,-0.076362,0.007896
3748,2020-12-28,106.804451,13.747366,183.627274,21.700001,0.000170,0.000169,-0.006164,0.002757,0.006327,-0.004174,0.007896,0.063594


In [ ]:
data_na.head(10)

In [5]:
def create_batches(dataframe, start_date, end_date, window_size=50):
    dataframe['Date'] = pd.to_datetime(dataframe['Date'])
    
    filtered_data = dataframe[(dataframe['Date'] >= start_date) & (dataframe['Date'] <= end_date)]
    
    x_batches = []
    y_batches = []
    dates_batches = []
    dates_per_feature = []
    y = []
    
    for i in range(len(filtered_data) - window_size):
        window_returns = filtered_data.iloc[i:i+window_size][[col for col in dataframe.columns if col.endswith('_R')]].values
        window_prices = filtered_data.iloc[i:i+window_size][[col for col in dataframe.columns if col in ['AGG', 'DBC', 'VTI', '^VIX']]].values
        window_x = np.concatenate([window_returns, window_prices], axis=1)
        window_y = filtered_data.iloc[i:i+window_size][[col for col in dataframe.columns if col.endswith('_y')]].values
        
        window_dates_batches = filtered_data.iloc[i:i+window_size]['Date'].values
        dates_per_feature.append(filtered_data.iloc[i+window_size]['Date'].date()) 

        y.append(filtered_data.iloc[i+window_size][[col for col in dataframe.columns if col.endswith('_y')]].values)

        x_batches.append(window_x)
        y_batches.append(window_y)
        dates_batches.append(window_dates_batches)

    return np.array(x_batches), np.array(y_batches), np.array(y, dtype=np.float32), np.array(dates_batches), dates_per_feature

x_batches, y_batches, y, dates_batches, dates_per_feature = create_batches(data_na, start_date="2006-01-01", end_date="2020-12-29")
y = y.reshape(y.shape[0], 1, y.shape[1])

#dates_per_feature représente la date à laquelle la valeur de la cible (dans y) est observée: c les 51èmes valeurs donc la valeur qui suit chaque window de 50 points

C:\Users\tahah\AppData\Local\Temp\ipykernel_10784\2991668214.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Date'] = pd.to_datetime(dataframe['Date'])


In [ ]:
print(x_batches.shape)

In [ ]:
x_batches.shape

In [ ]:
len(dates_per_feature)

In [ ]:
data_na.iloc[10]['Date'].date()

In [ ]:
data_na.iloc[:10]['Date'].values


In [ ]:
window_returns = data_na.iloc[:10][['AGG_R', 'DBC_R']].values
window_prices = data_na.iloc[:10][[col for col in data_na.columns if col in ['AGG', 'DBC']]].values

window_returns.shape, window_prices.shape

concatenaed = np.concatenate([window_returns, window_prices], axis=1)
concatenaed.shape

In [6]:
import datetime


def get_idx_dates(start_date, end_date):
    if start_date in dates_per_feature and end_date in dates_per_feature:
        k_start = dates_per_feature.index(start_date)
        k_end = dates_per_feature.index(end_date)
        return k_start, k_end
    
    if start_date not in dates_per_feature: 
        if any(dates >= start_date for dates in dates_per_feature):    
            k_start = min(j for j in range(len(dates_per_feature)) if dates_per_feature[j] >= start_date)
        else:
            k_start = 0
    else:
        k_start = dates_per_feature.index(start_date)

    if end_date not in dates_per_feature:
        if any(dates <= end_date for dates in dates_per_feature):
            k_end = max(j for j in range(len(dates_per_feature)) if dates_per_feature[j] <= end_date) + 1
        else:
            k_end = len(dates_per_feature) -1
    else: 
        k_end = dates_per_feature.index(end_date)

    return k_start, k_end


def training(x_batches, y_batches, model, batch_size):

    x_batches_tensor = torch.tensor(x_batches, dtype=torch.float32)
    y_batches_tensor = torch.tensor(y_batches, dtype=torch.float32)
    num_batches = x_batches.shape[0]
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 100


    for epoch in range(num_epochs):
        model.train()
        loss_epoch = 0
        
        for i in range(0, num_batches, batch_size):
            x_mini_batch = x_batches_tensor[i:i+batch_size]
            y_mini_batch = y_batches_tensor[i:i+batch_size]
            
            optimizer.zero_grad()
            
            loss = model(x_mini_batch, y_mini_batch)
            loss_epoch += loss.item()
            loss.backward()
            optimizer.step()
        
        print(f"epoch [{epoch+1}/{num_epochs}], loss: {(loss_epoch/batch_size)}")

    return model



def investing(x_batches, y, model):

    x_tensor = torch.tensor(x_batches, dtype=torch.float32)
    allocations = model.get_allocations(x_tensor)
    # pourquoi ce shape ?
    allocations = allocations.view(allocations.shape[0], 1, allocations.shape[1])
    rdt = torch.sum(allocations*y, dim=2)

    # return du portfeuille à t final du rnn, les allocations
    return rdt, allocations


In [7]:
input_size = 8
hidden_size = 64
output_size = 4
batch_size = 64

rdt_all = torch.empty(0,0)
alloc_all = []
idx_invest = []
dates_invest = []

# first training goes from 2006 to 2010
first_date = dates_per_feature[0]
end_date_1st_training = first_date + datetime.timedelta(days=365*4)

# trouve le premier batch et le dernier batch correspondant
idx_start, idx_end = get_idx_dates(first_date, end_date_1st_training)

model = PortfolioRNN(input_size=input_size, hidden_size=hidden_size, num_layers = 2,output_size=output_size)


model = training(x_batches = x_batches[idx_start:idx_end, :, :],
                      y_batches= y_batches[idx_start:idx_end, :, :],
                      model=model,
                      batch_size=64)

# un an c pas 365 jours mais 252 jours de trading
date_end_invest = end_date_1st_training + datetime.timedelta(days=365*2)
idx_start_invest, idx_end_invest = get_idx_dates(end_date_1st_training, date_end_invest)

print(f"Idx start: {idx_start_invest}, Idx end: {idx_end_invest}")
print(dates_per_feature[idx_start_invest], dates_per_feature[idx_end_invest])

idx_invest.append((idx_start_invest, idx_end_invest))
dates_invest.append((end_date_1st_training, date_end_invest))

x_investing = x_batches[idx_start_invest:idx_end_invest, :, :]
y_investing = y_batches[idx_start_invest:idx_end_invest, :, :]
y_real = y[idx_start_invest:idx_end_invest, :, :]

first_rdt, first_alloc = investing(x_batches= x_investing, y=y_real, model=model)

print(first_rdt.shape, len(dates_per_feature[idx_start_invest:idx_end_invest]))

print(f"first_rdt shape: {first_rdt.shape}, rdt_all shape before concat: {rdt_all.shape}")



epoch [1/100], loss: -0.029385375937636127
epoch [2/100], loss: -0.03307000907079782
epoch [3/100], loss: -0.04018617179826833
epoch [4/100], loss: -0.03156034466519486
epoch [5/100], loss: -0.040701317571802065
epoch [6/100], loss: -0.03472963490639813
epoch [7/100], loss: -0.043660113915393595
epoch [8/100], loss: -0.038916635798159405
epoch [9/100], loss: -0.04196471803879831
epoch [10/100], loss: -0.044280828136834316
epoch [11/100], loss: -0.04768573050387204
epoch [12/100], loss: -0.041050448548048735
epoch [13/100], loss: -0.03955679957289249
epoch [14/100], loss: -0.032119931229317444
epoch [15/100], loss: -0.04176261443353724
epoch [16/100], loss: -0.04353887538309209
epoch [17/100], loss: -0.05077271064510569
epoch [18/100], loss: -0.048479116576345405
epoch [19/100], loss: -0.05474630840035388
epoch [20/100], loss: -0.05331587987893727
epoch [21/100], loss: -0.056429067939461675
epoch [22/100], loss: -0.05400195966285537
epoch [23/100], loss: -0.054336018365575
epoch [24/100

In [ ]:
print(first_alloc.shape)

In [8]:

rdt_all = first_rdt
alloc_all = first_alloc

#entrainement sur 5 cycles Cycle 1 :
#Période d’entraînement : 2006 à 2010.
#Période d’investissement : 2010 à 2012.
#Cycle 2 :
#La date de fin d’entraînement est avancée jusqu'en 2012 (couvrant 2006-2012).
#La nouvelle période d’investissement couvre alors 2012 à 2014.
#etc jusqu'au cycle 5 
#rdt_all/alloc_all contient les rendements/alloc sur toute la période 

#change tp 4
for i in range(5):

    model = training(x_batches = x_batches[idx_start_invest:idx_end_invest, :, :],
                      y_batches= y_batches[idx_start_invest:idx_end_invest, :, :],
                      model=model,
                      batch_size=64)
    
    date_end_invest = date_end_invest + datetime.timedelta(days=365*2)
    end_date_1st_training = end_date_1st_training + datetime.timedelta(days=365*2)
    idx_start_invest, idx_end_invest = get_idx_dates(end_date_1st_training, date_end_invest)

    x_investing = x_batches[idx_start_invest:idx_end_invest, :, :]
    y_investing = y_batches[idx_start_invest:idx_end_invest, :, :]
    y_real = y[idx_start_invest:idx_end_invest, :, :]


    first_rdt, first_alloc = investing(x_batches= x_investing, y=y_real, model=model)

    print(first_rdt.shape, len(dates_per_feature[idx_start_invest:idx_end_invest]))


    rdt_all = torch.cat([rdt_all, first_rdt], dim=0)
    print('len des rdt', rdt_all.shape[0])
    alloc_all = torch.cat([alloc_all, first_alloc], dim=0)
    idx_invest.append((idx_start_invest, idx_end_invest))
    print('len des dates', len(dates_per_feature[idx_invest[0][0]:idx_invest[-1][1]]))
    dates_invest.append((end_date_1st_training, date_end_invest))


alloc_all = alloc_all.numpy()
alloc_all = alloc_all.reshape(alloc_all.shape[0], alloc_all.shape[2])

epoch [1/100], loss: -0.024832538794726133
epoch [2/100], loss: -0.034344499465078115
epoch [3/100], loss: -0.038555337116122246
epoch [4/100], loss: -0.04193546692840755
epoch [5/100], loss: -0.043892650632187724
epoch [6/100], loss: -0.04602160467766225
epoch [7/100], loss: -0.04618793702684343
epoch [8/100], loss: -0.047210816061124206
epoch [9/100], loss: -0.04887378145940602
epoch [10/100], loss: -0.048813041765242815
epoch [11/100], loss: -0.0489580393768847
epoch [12/100], loss: -0.04846715950407088
epoch [13/100], loss: -0.05027770507149398
epoch [14/100], loss: -0.050180085469037294
epoch [15/100], loss: -0.052126366179436445
epoch [16/100], loss: -0.052794078597798944
epoch [17/100], loss: -0.055410209111869335
epoch [18/100], loss: -0.05309193045832217
epoch [19/100], loss: -0.054598760325461626
epoch [20/100], loss: -0.054940992500633
epoch [21/100], loss: -0.05780458124354482
epoch [22/100], loss: -0.05372153199277818
epoch [23/100], loss: -0.05369267472997308
epoch [24/10

In [9]:

dataframe_result = pd.DataFrame({
    'Date': pd.Series(dates_per_feature[idx_invest[0][0]:idx_invest[-1][1]]),
    'Return': pd.Series(rdt_all.numpy().flatten()),
    'Alloc_AGG': pd.Series(alloc_all[:, 0]),
    'Alloc_DBC': pd.Series(alloc_all[:, 1]),
    'Alloc_VTI': pd.Series(alloc_all[:, 2]),
    'Alloc_^VIX': pd.Series(alloc_all[:, 3])
})

dataframe_result['Date'] = pd.to_datetime(dataframe_result['Date'])
data_na['Date'] = pd.to_datetime(data_na['Date'])
dataframe_result = pd.merge(dataframe_result, data_na[['Date', 'AGG_y', 'DBC_y', 'VTI_y', '^VIX_y']], on='Date', how='left')

dataframe_result['Final_Return'] = (
    dataframe_result['Alloc_AGG'] * dataframe_result['AGG_y'] +
    dataframe_result['Alloc_DBC'] * dataframe_result['DBC_y'] +
    dataframe_result['Alloc_VTI'] * dataframe_result['VTI_y'] +
    dataframe_result['Alloc_^VIX'] * dataframe_result['^VIX_y']
)

dataframe_result

C:\Users\tahah\AppData\Local\Temp\ipykernel_10784\2977585624.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_na['Date'] = pd.to_datetime(data_na['Date'])


,Date,Return,Alloc_AGG,Alloc_DBC,Alloc_VTI,Alloc_^VIX,AGG_y,DBC_y,VTI_y,^VIX_y,Final_Return
0,2010-04-19,0.001405,0.552563,0.003101,0.407883,0.036454,0.001631,0.007936,0.009474,-0.092849,0.001405
1,2010-04-20,0.001786,0.769451,0.002737,0.211385,0.016427,0.001724,0.004973,-0.000809,0.037508,0.001786
2,2010-04-21,-0.000540,0.808225,0.005197,0.148870,0.037709,-0.002008,0.002474,0.004858,0.009191,-0.000540
3,2010-04-22,0.006023,0.146700,0.005446,0.663633,0.184221,-0.001246,0.006582,0.006769,0.009108,0.006023
4,2010-04-23,0.001346,0.272422,0.004133,0.651372,0.072072,-0.000096,-0.004904,-0.003521,0.051143,0.001346
...,...,...,...,...,...,...,...,...,...,...,...
2690,2020-12-22,-0.000645,0.958990,0.001089,0.037993,0.001928,-0.000678,0.012517,0.001710,-0.037969,-0.000645
2691,2020-12-23,0.000713,0.970023,0.000880,0.025979,0.003118,0.000933,0.002747,0.001673,-0.076362,0.000713
2692,2020-12-24,0.000934,0.874259,0.002577,0.108709,0.014454,0.000170,-0.006164,0.006327,0.007896,0.000934
2693,2020-12-28,0.000851,0.948678,0.000308,0.037697,0.013318,0.000169,0.002757,-0.004174,0.063594,0.000851


In [10]:
trading_days_per_year = 252

mean_daily_return = dataframe_result['Final_Return'].mean()
annualized_return = (1+mean_daily_return) ** trading_days_per_year -1

std_daily_return = dataframe_result['Final_Return'].std()
std_dd_return = dataframe_result[dataframe_result['Final_Return'] < 0]['Final_Return'].std()

annualized_volatility = std_daily_return * np.sqrt(trading_days_per_year)
annualized_volatility_dd = std_dd_return * np.sqrt(trading_days_per_year)

sharpe_ratio = annualized_return / annualized_volatility
sortino_ratio = annualized_return / annualized_volatility_dd

print(f"annualized mean return: {annualized_return:.4f}")
print(f"annualized std: {annualized_volatility:.4f}")
print(f"annualized dd: {annualized_volatility_dd:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"Sortino Ratio: {sortino_ratio:.4f}")

annualized mean return: 0.3455
annualized std: 0.2630
annualized dd: 0.1250
Sharpe Ratio: 1.3138
Sortino Ratio: 2.7634


# Volatilities 

In [11]:
def calculate_volatility(x_batches, decay=0.94):
    """
    Calculate the ex-ante volatility for each asset in each window in x_batches
    using an exponentially weighted moving standard deviation.
    
    Parameters:
    - x_batches: torch.Tensor of shape (num_windows, window_size, num_features)
    - decay: float, decay factor for exponential weighting (e.g., 0.94)
    
    Returns:
    - volatilities: torch.Tensor of shape (num_windows, num_assets)
    """
    # Extract only the return columns; x_batches has shape (num_windows, window_size=50, num_features=8)
    returns_batches = x_batches[:, :, :x_batches.shape[2] // 2]
    window_size = returns_batches.shape[1]
    num_assets = returns_batches.shape[2]
    
    weights = torch.tensor([decay ** i for i in range(window_size - 1, -1, -1)], dtype=torch.float32)
    weights /= weights.sum()  
    
    volatilities = []
    for window in returns_batches:
        window_tensor = torch.tensor(window, dtype=torch.float32)  # Convert `window` to a tensor
        weighted_returns = window_tensor * weights.view(-1, 1)  # Apply weights
        mean_weighted = weighted_returns.sum(dim=0)  # Weighted mean for each asset
        variance_weighted = ((window_tensor - mean_weighted) ** 2 * weights.view(-1, 1)).sum(dim=0)
        std_weighted = torch.sqrt(variance_weighted)
        volatilities.append(std_weighted)
    
    # Stack for all windows
    volatilities = torch.stack(volatilities)
    return volatilities  # We have the volatilities at the final date for each 50-day window


In [12]:
def investing_scaled(x_batches, y, model, volatilities, target_volatility=0.1, cost_rate=0.0001): 
    """
    Calculates the portfolio returns using model allocations, adjusted by volatility scaling and transaction costs.

    Parameters:
    - x_batches: np.array, shape (num_batches, window_size, input_size), input data (returns and prices).
    - y: np.array, shape (num_batches, 1, num_assets), target returns for each day in the investing period.
    - model: trained model for generating portfolio allocations.
    - volatilities: np.array, shape (num_batches, num_assets), ex-ante volatilities for each asset.
    - target_volatility: float, target volatility level.
    - cost_rate: float, transaction cost rate.

    Returns:
    - portfolio_returns: Tensor, shape (num_batches, 1), calculated portfolio returns.
    - allocations: Tensor, shape (num_batches, num_assets), model's allocations for each asset.
    """
    x_tensor = torch.tensor(x_batches, dtype=torch.float32)
    allocations = model.get_allocations(x_tensor)  # Shape: (num_batches, num_assets)

    allocations = allocations.view(allocations.shape[0], 1, allocations.shape[1])
    
    volatilities = torch.tensor(volatilities, dtype=torch.float32)
    
    scaling_factor = target_volatility / volatilities  # Shape: (num_batches, num_assets)
    scaled_allocations = allocations * scaling_factor.unsqueeze(1) 

    # on décale de 1 à la prem dimension car c celle du temps 
    transaction_costs = cost_rate * torch.sum(torch.abs(scaled_allocations[1:] - scaled_allocations[:-1]), dim=2)


    portfolio_returns = torch.sum(scaled_allocations * y, dim=2)
    portfolio_returns[1:] -= transaction_costs  # Apply transaction costs from the second time step onward

    return portfolio_returns, scaled_allocations


In [13]:
input_size = 8
hidden_size = 64
output_size = 4
batch_size = 64

rdt_all = torch.empty(0,0)
alloc_all = []
idx_invest = []
dates_invest = []

# first training goes from 2006 to 2010
first_date = dates_per_feature[0]
end_date_1st_training = first_date + datetime.timedelta(days=365*4)

# trouve le premier batch et le dernier batch correspondant
idx_start, idx_end = get_idx_dates(first_date, end_date_1st_training)

model = PortfolioRNN(input_size=input_size, hidden_size=hidden_size, num_layers = 2,output_size=output_size)


model = training(x_batches = x_batches[idx_start:idx_end, :, :],
                      y_batches= y_batches[idx_start:idx_end, :, :],
                      model=model,
                      batch_size=64)

# un an c pas 365 jours mais 252 jours de trading
date_end_invest = end_date_1st_training + datetime.timedelta(days=365*2)
idx_start_invest, idx_end_invest = get_idx_dates(end_date_1st_training, date_end_invest)

print(f"Idx start: {idx_start_invest}, Idx end: {idx_end_invest}")
print(dates_per_feature[idx_start_invest], dates_per_feature[idx_end_invest])

idx_invest.append((idx_start_invest, idx_end_invest))
dates_invest.append((end_date_1st_training, date_end_invest))

x_investing = x_batches[idx_start_invest:idx_end_invest, :, :]
y_investing = y_batches[idx_start_invest:idx_end_invest, :, :]
y_real = y[idx_start_invest:idx_end_invest, :, :]

#calcul des volatilités
volatilities = calculate_volatility(x_batches=x_investing)

first_rdt, first_alloc = investing_scaled(x_batches= x_investing, y=y_real, volatilities=volatilities, model=model)

print(first_rdt.shape, len(dates_per_feature[idx_start_invest:idx_end_invest]))

print(f"first_rdt shape: {first_rdt.shape}, rdt_all shape before concat: {rdt_all.shape}")



epoch [1/100], loss: -0.025247834506444633
epoch [2/100], loss: -0.03462966068764217
epoch [3/100], loss: -0.03978791073313914
epoch [4/100], loss: -0.03442962621920742
epoch [5/100], loss: -0.03988870786270127
epoch [6/100], loss: -0.035906557575799525
epoch [7/100], loss: -0.04351254637003876
epoch [8/100], loss: -0.03881423383427318
epoch [9/100], loss: -0.04347448970656842
epoch [10/100], loss: -0.04298895166721195
epoch [11/100], loss: -0.046827532758470625
epoch [12/100], loss: -0.046222640994528774
epoch [13/100], loss: -0.04213450910174288
epoch [14/100], loss: -0.05186720700294245
epoch [15/100], loss: -0.05302463566476945
epoch [16/100], loss: -0.05362520504422719
epoch [17/100], loss: -0.0550864093893324
epoch [18/100], loss: -0.05650403333129361
epoch [19/100], loss: -0.05540634581120685
epoch [20/100], loss: -0.05359545738610905
epoch [21/100], loss: -0.049116382768261246
epoch [22/100], loss: -0.04435647831269307
epoch [23/100], loss: -0.05404937075218186
epoch [24/100], 

C:\Users\tahah\AppData\Local\Temp\ipykernel_10784\3173161383.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  volatilities = torch.tensor(volatilities, dtype=torch.float32)


In [14]:

rdt_all = first_rdt
alloc_all = first_alloc

#entrainement sur 5 cycles Cycle 1 :
#Période d’entraînement : 2006 à 2010.
#Période d’investissement : 2010 à 2012.
#Cycle 2 :
#La date de fin d’entraînement est avancée jusqu'en 2012 (couvrant 2006-2012).
#La nouvelle période d’investissement couvre alors 2012 à 2014.
#etc jusqu'au cycle 5 
#rdt_all/alloc_all contient les rendements/alloc sur toute la période 

#change tp 4
for i in range(5):

    model = training(x_batches = x_batches[idx_start_invest:idx_end_invest, :, :],
                      y_batches= y_batches[idx_start_invest:idx_end_invest, :, :],
                      model=model,
                      batch_size=64)
    
    date_end_invest = date_end_invest + datetime.timedelta(days=365*2)
    end_date_1st_training = end_date_1st_training + datetime.timedelta(days=365*2)
    idx_start_invest, idx_end_invest = get_idx_dates(end_date_1st_training, date_end_invest)

    x_investing = x_batches[idx_start_invest:idx_end_invest, :, :]
    y_investing = y_batches[idx_start_invest:idx_end_invest, :, :]
    y_real = y[idx_start_invest:idx_end_invest, :, :]

    #calcul des volatilités
    volatilities = calculate_volatility(x_batches=x_investing)

    first_rdt, first_alloc = investing_scaled(x_batches= x_investing, y=y_real, volatilities=volatilities, model=model)

    print(first_rdt.shape, len(dates_per_feature[idx_start_invest:idx_end_invest]))


    rdt_all = torch.cat([rdt_all, first_rdt], dim=0)
    print('len des rdt', rdt_all.shape[0])
    alloc_all = torch.cat([alloc_all, first_alloc], dim=0)
    idx_invest.append((idx_start_invest, idx_end_invest))
    print('len des dates', len(dates_per_feature[idx_invest[0][0]:idx_invest[-1][1]]))
    dates_invest.append((end_date_1st_training, date_end_invest))


alloc_all = alloc_all.numpy()
alloc_all = alloc_all.reshape(alloc_all.shape[0], alloc_all.shape[2])

epoch [1/100], loss: -0.027060609310865402
epoch [2/100], loss: -0.03377311024814844
epoch [3/100], loss: -0.036357807694002986
epoch [4/100], loss: -0.037508981535211205
epoch [5/100], loss: -0.038861619075760245
epoch [6/100], loss: -0.039881942328065634
epoch [7/100], loss: -0.041656311601400375
epoch [8/100], loss: -0.04213876649737358
epoch [9/100], loss: -0.04376630438491702
epoch [10/100], loss: -0.045357906725257635
epoch [11/100], loss: -0.046239478047937155
epoch [12/100], loss: -0.047415432054549456
epoch [13/100], loss: -0.04764600354246795
epoch [14/100], loss: -0.04682529345154762
epoch [15/100], loss: -0.04737794864922762
epoch [16/100], loss: -0.04666434554383159
epoch [17/100], loss: -0.04980685259215534
epoch [18/100], loss: -0.049754428677260876
epoch [19/100], loss: -0.050867271842435
epoch [20/100], loss: -0.0483173334505409
epoch [21/100], loss: -0.04487251280806959
epoch [22/100], loss: -0.04765549302101135
epoch [23/100], loss: -0.04486356978304684
epoch [24/100

C:\Users\tahah\AppData\Local\Temp\ipykernel_10784\3173161383.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  volatilities = torch.tensor(volatilities, dtype=torch.float32)


epoch [1/100], loss: -0.014108190167462453
epoch [2/100], loss: -0.027856891276314855
epoch [3/100], loss: -0.03222095454111695
epoch [4/100], loss: -0.03262593410909176
epoch [5/100], loss: -0.03791768755763769
epoch [6/100], loss: -0.03951485990546644
epoch [7/100], loss: -0.04032116383314133
epoch [8/100], loss: -0.04280895437113941
epoch [9/100], loss: -0.04388297419063747
epoch [10/100], loss: -0.045038087759166956
epoch [11/100], loss: -0.04564559133723378
epoch [12/100], loss: -0.0461807104293257
epoch [13/100], loss: -0.046416802098974586
epoch [14/100], loss: -0.047229107236489654
epoch [15/100], loss: -0.045946171041578054
epoch [16/100], loss: -0.042000594548881054
epoch [17/100], loss: -0.04086728685069829
epoch [18/100], loss: -0.044985092943534255
epoch [19/100], loss: -0.047675623558461666
epoch [20/100], loss: -0.04949875548481941
epoch [21/100], loss: -0.049709733575582504
epoch [22/100], loss: -0.05006291624158621
epoch [23/100], loss: -0.048559530172497034
epoch [24/

In [15]:

dataframe_result = pd.DataFrame({
    'Date': pd.Series(dates_per_feature[idx_invest[0][0]:idx_invest[-1][1]]),
    'Return': pd.Series(rdt_all.numpy().flatten()),
    'Alloc_AGG': pd.Series(alloc_all[:, 0]),
    'Alloc_DBC': pd.Series(alloc_all[:, 1]),
    'Alloc_VTI': pd.Series(alloc_all[:, 2]),
    'Alloc_^VIX': pd.Series(alloc_all[:, 3])
})

dataframe_result['Date'] = pd.to_datetime(dataframe_result['Date'])
data_na['Date'] = pd.to_datetime(data_na['Date'])
dataframe_result = pd.merge(dataframe_result, data_na[['Date', 'AGG_y', 'DBC_y', 'VTI_y', '^VIX_y']], on='Date', how='left')

dataframe_result['Final_Return'] = (
    dataframe_result['Alloc_AGG'] * dataframe_result['AGG_y'] +
    dataframe_result['Alloc_DBC'] * dataframe_result['DBC_y'] +
    dataframe_result['Alloc_VTI'] * dataframe_result['VTI_y'] +
    dataframe_result['Alloc_^VIX'] * dataframe_result['^VIX_y']
)

dataframe_result

C:\Users\tahah\AppData\Local\Temp\ipykernel_10784\2977585624.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_na['Date'] = pd.to_datetime(data_na['Date'])


,Date,Return,Alloc_AGG,Alloc_DBC,Alloc_VTI,Alloc_^VIX,AGG_y,DBC_y,VTI_y,^VIX_y,Final_Return
0,2010-04-19,0.094613,19.772177,0.211438,7.343978,0.095791,0.001631,0.007936,0.009474,-0.092849,0.094613
1,2010-04-20,0.053121,32.027927,0.144910,4.324626,0.058925,0.001724,0.004973,-0.000809,0.037508,0.054659
2,2010-04-21,-0.075321,41.022823,0.213873,1.463578,0.064303,-0.002008,0.002474,0.004858,0.009191,-0.074128
3,2010-04-22,0.070810,4.960678,1.045565,10.788657,0.189071,-0.001246,0.006582,0.006769,0.009108,0.075445
4,2010-04-23,-0.045754,2.896227,0.188570,13.922251,0.099532,-0.000096,-0.004904,-0.003521,0.051143,-0.045139
...,...,...,...,...,...,...,...,...,...,...,...
2690,2020-12-22,-0.034075,53.899368,0.003227,1.639510,0.003607,-0.000678,0.012517,0.001710,-0.037969,-0.033858
2691,2020-12-23,0.057148,62.520584,0.000121,0.074403,0.004039,0.000933,0.002747,0.001673,-0.076362,0.058167
2692,2020-12-24,0.024314,51.077675,0.001347,2.692183,0.004066,0.000170,-0.006164,0.006327,0.007896,0.025720
2693,2020-12-28,0.000823,49.894943,0.015985,2.758587,0.062451,0.000169,0.002757,-0.004174,0.063594,0.000955


In [16]:
trading_days_per_year = 252

mean_daily_return = dataframe_result['Final_Return'].mean()
annualized_return = (1+mean_daily_return) ** trading_days_per_year -1

std_daily_return = dataframe_result['Final_Return'].std()
std_dd_return = dataframe_result[dataframe_result['Final_Return'] < 0]['Final_Return'].std()

annualized_volatility = std_daily_return * np.sqrt(trading_days_per_year)
annualized_volatility_dd = std_dd_return * np.sqrt(trading_days_per_year)

sharpe_ratio = annualized_return / annualized_volatility
sortino_ratio = annualized_return / annualized_volatility_dd

print(f"annualized mean return: {annualized_return:.4f}")
print(f"annualized std: {annualized_volatility:.4f}")
print(f"annualized dd: {annualized_volatility_dd:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"Sortino Ratio: {sortino_ratio:.4f}")

annualized mean return: 5.1104
annualized std: 1.2098
annualized dd: 0.9341
Sharpe Ratio: 4.2241
Sortino Ratio: 5.4711
